In [17]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
import joblib
import time
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')

# ML Libraries
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import TimeSeriesSplit, train_test_split
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier

# Paths
BASE_DIR = Path(r'c:\Users\Acer\Desktop\Forex-Signal-App')
DATA_DIR = BASE_DIR / 'data'
MODEL_DIR = BASE_DIR / 'models' / 'signal_generator_v16'
MODEL_DIR.mkdir(parents=True, exist_ok=True)

# Constants
TIMEFRAMES = ['1min', '5min', '15min', '30min', '1h']
TARGET_TIMEFRAME = '1min'  # We predict on 1-min data using MTF context
FORWARD_BARS = 15          # Predict 15 minutes into the future
TP_PIPS = 10
SL_PIPS = 10

print(f"🚀 FOREX SIGNAL V16: Multi-Timeframe Analysis (Combined Data)")
print(f"✓ Model Directory: {MODEL_DIR}")

🚀 FOREX SIGNAL V16: Multi-Timeframe Analysis (Combined Data)
✓ Model Directory: c:\Users\Acer\Desktop\Forex-Signal-App\models\signal_generator_v16


In [18]:
# 1. Load Data (Train & Test Separately)
def load_tf_data(timeframe, dataset_type='train'):
    """
    Load specific timeframe data from train or test folder
    """
    filename = f'EURUSD_{timeframe}.csv'
    file_path = DATA_DIR / dataset_type / filename
        
    if not file_path.exists():
        print(f"⚠️ Warning: {file_path} not found.")
        return None
        
    df = pd.read_csv(file_path)
    
    # Standardize columns
    df.columns = [c.lower() for c in df.columns]
    
    # Ensure time column is datetime
    if 'time' in df.columns:
        df['time'] = pd.to_datetime(df['time'])
        df.set_index('time', inplace=True)
        df.sort_index(inplace=True)
    
    return df

timeframes_map = {
    '1min': 'm1',
    '5min': 'm5',
    '15min': 'm15',
    '30min': 'm30',
    '1h': 'h1',
    '4h': 'h4'
}

dfs_train = {}
dfs_test = {}

print("Loading TRAIN data...")
for tf_name, tf_code in timeframes_map.items():
    dfs_train[tf_name] = load_tf_data(tf_code, 'train')
    if dfs_train[tf_name] is not None:
        print(f"  {tf_name}: {len(dfs_train[tf_name]):,} rows")

print("\nLoading TEST data...")
for tf_name, tf_code in timeframes_map.items():
    dfs_test[tf_name] = load_tf_data(tf_code, 'test')
    if dfs_test[tf_name] is not None:
        print(f"  {tf_name}: {len(dfs_test[tf_name]):,} rows")

Loading TRAIN data...
  1min: 3,354,904 rows
  5min: 671,581 rows
  15min: 224,382 rows
  30min: 112,194 rows
  1h: 56,098 rows
  4h: 14,498 rows

Loading TEST data...
  1min: 743,476 rows
  5min: 148,502 rows
  15min: 49,807 rows
  30min: 24,907 rows
  1h: 12,454 rows
  4h: 3,220 rows


In [19]:
# 2. Feature Engineering
def add_technical_indicators(df):
    """
    Calculates technical indicators for a given dataframe.
    """
    if df is None or len(df) == 0:
        return None
        
    df = df.copy()
    
    # RSI (14)
    delta = df['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / (loss + 1e-10)
    df['rsi'] = 100 - (100 / (1 + rs))
    
    # MACD (12, 26, 9)
    ema12 = df['close'].ewm(span=12, adjust=False).mean()
    ema26 = df['close'].ewm(span=26, adjust=False).mean()
    df['macd'] = ema12 - ema26
    df['macd_signal'] = df['macd'].ewm(span=9, adjust=False).mean()
    df['macd_hist'] = df['macd'] - df['macd_signal']
    
    # Bollinger Bands (20, 2)
    df['bb_mid'] = df['close'].rolling(window=20).mean()
    df['bb_std'] = df['close'].rolling(window=20).std()
    df['bb_upper'] = df['bb_mid'] + (df['bb_std'] * 2)
    df['bb_lower'] = df['bb_mid'] - (df['bb_std'] * 2)
    df['bb_width'] = (df['bb_upper'] - df['bb_lower']) / df['bb_mid']
    
    # EMAs
    for span in [9, 21, 50, 200]:
        df[f'ema_{span}'] = df['close'].ewm(span=span, adjust=False).mean()
        
    # ATR (14)
    high_low = df['high'] - df['low']
    high_close = (df['high'] - df['close'].shift()).abs()
    low_close = (df['low'] - df['close'].shift()).abs()
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = ranges.max(axis=1)
    df['atr'] = true_range.rolling(14).mean()
    
    return df.dropna()

def process_all_timeframes(dfs_dict):
    processed = {}
    for tf, df in dfs_dict.items():
        if df is not None:
            # print(f"Processing {tf}...")
            processed[tf] = add_technical_indicators(df)
            
            # No renaming needed for independent models
            # We keep original column names (open, high, low, close) 
            # so create_target can find them easily.
            
    return processed

print("Processing TRAIN features...")
processed_train = process_all_timeframes(dfs_train)

print("Processing TEST features...")
processed_test = process_all_timeframes(dfs_test)

print("Feature engineering complete.")

Processing TRAIN features...
Processing TEST features...
Feature engineering complete.


In [20]:
# 3. Define Target (Per Timeframe)
def create_target(df, tp_pips=10, sl_pips=10, forward_bars=15):
    targets = []
    pip_size = 0.0001
    tp = tp_pips * pip_size
    sl = sl_pips * pip_size
    
    closes = df['close'].values
    highs = df['high'].values
    lows = df['low'].values
    
    for i in range(len(df) - forward_bars):
        current_close = closes[i]
        
        # Look ahead window
        future_highs = highs[i+1 : i+1+forward_bars]
        future_lows = lows[i+1 : i+1+forward_bars]
        
        max_high = np.max(future_highs)
        min_low = np.min(future_lows)
        
        # Check for Buy Signal
        if (max_high - current_close) >= tp and (current_close - min_low) < sl:
            targets.append(1) # BUY
        # Check for Sell Signal
        elif (current_close - min_low) >= tp and (max_high - current_close) < sl:
            targets.append(2) # SELL
        else:
            targets.append(0) # HOLD
            
    # Pad the end
    targets.extend([0] * forward_bars)
    return np.array(targets)

print("Creating targets for ALL timeframes...")

# Apply to TRAIN
train_datasets = {}
for tf, df in processed_train.items():
    if df is not None and len(df) > FORWARD_BARS:
        print(f"  Creating target for TRAIN {tf}...")
        df = df.copy()
        df['target'] = create_target(df, TP_PIPS, SL_PIPS, FORWARD_BARS)
        train_datasets[tf] = df.iloc[:-FORWARD_BARS] # Remove last rows

# Apply to TEST
test_datasets = {}
for tf, df in processed_test.items():
    if df is not None and len(df) > FORWARD_BARS:
        print(f"  Creating target for TEST {tf}...")
        df = df.copy()
        df['target'] = create_target(df, TP_PIPS, SL_PIPS, FORWARD_BARS)
        test_datasets[tf] = df.iloc[:-FORWARD_BARS]

print("Target creation complete.")

Creating targets for ALL timeframes...
  Creating target for TRAIN 1min...
  Creating target for TRAIN 5min...
  Creating target for TRAIN 15min...
  Creating target for TRAIN 30min...
  Creating target for TRAIN 1h...
  Creating target for TRAIN 4h...
  Creating target for TEST 1min...
  Creating target for TEST 5min...
  Creating target for TEST 15min...
  Creating target for TEST 30min...
  Creating target for TEST 1h...
  Creating target for TEST 4h...
Target creation complete.


In [21]:
# 4. Train Independent Models (Ensemble of Ensembles)
models = {}
scalers = {}
feature_cols_dict = {}

for tf, df_train in train_datasets.items():
    print(f"\nTraining Model for {tf}...")
    
    # Define features (exclude target and raw OHLC)
    cols = [c for c in df_train.columns if c not in ['target', 'open', 'high', 'low', 'close', 'tick_volume', 'spread', 'volume']]
    # We might want to keep the 'time' index, but it's the index.
    
    X_train = df_train[cols]
    y_train = df_train['target']
    
    # Scale
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    
    # Train XGBoost
    model = xgb.XGBClassifier(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=6,
        objective='multi:softprob',
        num_class=3,
        random_state=42,
        n_jobs=-1,
        verbosity=0
    )
    model.fit(X_train_scaled, y_train)
    
    models[tf] = model
    scalers[tf] = scaler
    feature_cols_dict[tf] = cols
    
    print(f"  ✓ {tf} Model Trained.")

print("\nAll models trained successfully.")


Training Model for 1min...
  ✓ 1min Model Trained.

Training Model for 5min...
  ✓ 5min Model Trained.

Training Model for 15min...
  ✓ 15min Model Trained.

Training Model for 30min...
  ✓ 30min Model Trained.

Training Model for 1h...
  ✓ 1h Model Trained.

Training Model for 4h...
  ✓ 4h Model Trained.

All models trained successfully.


In [27]:
# 5. Evaluation Functions
def get_predictions(models, scalers, datasets):
    preds_dict = {}
    for tf, df in datasets.items():
        if tf in models:
            # print(f"  Predicting {tf}...")
            model = models[tf]
            scaler = scalers[tf]
            cols = feature_cols_dict[tf]
            
            X = df[cols]
            X_scaled = scaler.transform(X)
            
            probs = model.predict_proba(X_scaled)
            preds_df = pd.DataFrame(probs, columns=['HOLD', 'BUY', 'SELL'], index=df.index)
            preds_dict[tf] = preds_df
    return preds_dict

def evaluate_ensemble(preds_dict, datasets, name="Set"):
    base_tf = '1min'
    if base_tf not in datasets:
        print(f"Error: 1min data needed for {name} evaluation.")
        return

    base_index = datasets[base_tf].index
    agg_probs = pd.DataFrame(0.0, index=base_index, columns=['HOLD', 'BUY', 'SELL'])
    
    count = 0
    for tf, preds_df in preds_dict.items():
        # Align to 1min
        # Forward fill to propagate higher timeframe signals to 1min bars
        aligned_preds = preds_df.reindex(base_index, method='ffill')
        agg_probs = agg_probs.add(aligned_preds, fill_value=0)
        count += 1
        
    final_probs = agg_probs / count
    final_classes = final_probs.idxmax(axis=1)
    
    class_map = {'HOLD': 0, 'BUY': 1, 'SELL': 2}
    y_pred = final_classes.map(class_map).fillna(0).astype(int)
    y_true = datasets[base_tf]['target'].reindex(base_index).fillna(0).astype(int)
    
    print(f"\n=== {name} Ensemble Performance ===")
    print(f"Overall Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(classification_report(y_true, y_pred))
    
    # --- Trade Only Evaluation ---
    # Filter where model predicted BUY (1) or SELL (2)
    trade_mask = y_pred != 0
    if trade_mask.sum() > 0:
        trade_preds = y_pred[trade_mask]
        trade_actuals = y_true[trade_mask]
        
        print(f"\n=== {name} Trade Only Performance (Excluding HOLD predictions) ===")
        print(f"Total Trades Predicted: {len(trade_preds)}")
        print(f"Trade Accuracy: {accuracy_score(trade_actuals, trade_preds):.4f}")
        
        # We are interested if the trade was correct. 
        # Note: If model predicts BUY(1) but actual is SELL(2), it's a loss.
        # If model predicts BUY(1) but actual is HOLD(0), it's a loss (false signal).
        
        print(classification_report(trade_actuals, trade_preds, zero_division=0))
    else:
        print(f"\n=== {name} Trade Only Performance ===")
        print("No trades predicted (All HOLD).")
    
    return y_true, y_pred

print("Evaluation functions defined.")

Evaluation functions defined.


In [28]:
# 6. Run Evaluation (Train vs Test)
print("Generating predictions for TRAINING set...")
train_preds = get_predictions(models, scalers, train_datasets)
evaluate_ensemble(train_preds, train_datasets, "TRAIN")

print("\nGenerating predictions for TEST set...")
test_preds = get_predictions(models, scalers, test_datasets)
evaluate_ensemble(test_preds, test_datasets, "TEST")

Generating predictions for TRAINING set...

=== TRAIN Ensemble Performance ===
Overall Accuracy: 0.8632
              precision    recall  f1-score   support

           0       0.87      1.00      0.93   2902360
           1       0.17      0.00      0.01    224491
           2       0.19      0.01      0.01    228019

    accuracy                           0.86   3354870
   macro avg       0.41      0.34      0.32   3354870
weighted avg       0.77      0.86      0.80   3354870


=== TRAIN Trade Only Performance (Excluding HOLD predictions) ===
Total Trades Predicted: 11831
Trade Accuracy: 0.1806
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      8527
           1       0.17      0.61      0.26      1606
           2       0.19      0.68      0.30      1698

    accuracy                           0.18     11831
   macro avg       0.12      0.43      0.19     11831
weighted avg       0.05      0.18      0.08     11831


Generating pr

(time
 2024-01-01 22:19:00    0
 2024-01-01 22:20:00    0
 2024-01-01 22:21:00    0
 2024-01-01 22:22:00    0
 2024-01-01 22:23:00    0
                       ..
 2025-12-30 23:40:00    0
 2025-12-30 23:41:00    0
 2025-12-30 23:42:00    0
 2025-12-30 23:43:00    0
 2025-12-30 23:44:00    0
 Name: target, Length: 743442, dtype: int64,
 time
 2024-01-01 22:19:00    0
 2024-01-01 22:20:00    0
 2024-01-01 22:21:00    0
 2024-01-01 22:22:00    0
 2024-01-01 22:23:00    0
                       ..
 2025-12-30 23:40:00    0
 2025-12-30 23:41:00    0
 2025-12-30 23:42:00    0
 2025-12-30 23:43:00    0
 2025-12-30 23:44:00    0
 Length: 743442, dtype: int64)

In [24]:
# 7. Save Models
print("Saving models...")
joblib.dump(models, MODEL_DIR / 'models_dict_v16.joblib')
joblib.dump(scalers, MODEL_DIR / 'scalers_dict_v16.joblib')
joblib.dump(feature_cols_dict, MODEL_DIR / 'feature_cols_dict_v16.joblib')

# Save config
config = {
    'timeframes': TIMEFRAMES,
    'tp_pips': TP_PIPS,
    'sl_pips': SL_PIPS,
    'forward_bars': FORWARD_BARS,
    'strategy': 'ensemble_of_ensembles'
}
joblib.dump(config, MODEL_DIR / 'config_v16.joblib')

print(f"Models saved to {MODEL_DIR}")

Saving models...
Models saved to c:\Users\Acer\Desktop\Forex-Signal-App\models\signal_generator_v16
